<a href="https://colab.research.google.com/github/johnathonpak/Laptronics-Analysis/blob/main/code/Laptronics_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Initialize and Load Dataframes**

In [3]:
import pandas as pd
from google.cloud import bigquery
from google.colab import auth


project_id = 'data-analysis-projects-456521'
dataset_id = 'data-analysis-projects-456521.laptronics_data'
table_id = [
    'data-analysis-projects-456521.laptronics_data.orders_raw',
    'data-analysis-projects-456521.laptronics_data.order_status',
    'data-analysis-projects-456521.laptronics_data.geo_lookup',
    'data-analysis-projects-456521.laptronics_data.customers'
]


client = bigquery.Client(project=project_id)
print(f"Client created using project: {client.project}")

Client created using project: data-analysis-projects-456521


In [4]:
# Authenticate Google Colab Enviro User
auth.authenticate_user()

print("Authenticated")

Authenticated


In [5]:
# Pull data into Dataframes

dfs = {}

for table_id in table_id:
    query = f'SELECT * FROM `{table_id}`'
    query_job = client.query(query)
    table_name = table_id.split('.')[-1]
    dfs[table_name] = query_job.to_dataframe()


df_orders_raw = dfs['orders_raw']
df_order_status = dfs['order_status']
df_geo_lookup = dfs['geo_lookup']
df_customers = dfs['customers']

print(df_orders_raw.head())
print(df_order_status.head())
print(df_geo_lookup.head())
print(df_customers.head())



    USER_ID     ORDER_ID PURCHASE_TS     SHIP_TS DELIVERY_TS REFUND_TS  \
0  c79310ae  68af2a1a9a2  11/14/2019  2019-11-16  2019-11-22       NaT   
1  fbebbfa6  2ad1c435942   2/23/2020  2020-02-24  2020-02-29       NaT   
2  337a5c9b  6fe8f2a20a5   1/30/2021  2021-02-02  2021-02-05       NaT   
3  90411b85  e8f07f649f5    7/6/2020  2020-07-07  2020-07-13       NaT   
4  adb0306e  eba06166e04   6/17/2021  2021-06-20  2021-06-27       NaT   

               PRODUCT_NAME PRODUCT_ID  USD_PRICE  LOCAL_PRICE CURRENCY  \
0  Apple Airpods Headphones       03ca     108.58         99.9      EUR   
1  Apple Airpods Headphones       03ca     119.90        119.9      USD   
2  Apple Airpods Headphones       03ca     119.90        119.9      USD   
3  Apple Airpods Headphones       03ca     119.90        119.9      USD   
4  Apple Airpods Headphones       03ca     119.90        119.9      USD   

  PURCHASE_PLATFORM MARKETING_CHANNEL ACCOUNT_CREATION_METHOD COUNTRY_CODE  \
0           website       

# **Data Cleaning**

In [14]:
# Duplicates Count on all tables
orders_duplicate = df_orders_raw.duplicated().sum()
print(f"Total number of duplicate rows for Orders: {orders_duplicate} \n")

order_status_duplicate = df_order_status.duplicated().sum()
print(f"Total number of duplicate rows for Order Status: {order_status_duplicate} \n")

geo_lookup_duplicate = df_geo_lookup.duplicated().sum()
print(f"Total number of duplicate rows for Geo Lookup: {geo_lookup_duplicate} \n")

customers_duplicate = df_customers.duplicated().sum()
print(f"Total number of duplicate rows for Customers: {customers_duplicate} \n")




Total number of duplicate rows for Orders: 15196 

Total number of duplicate rows for Order Status: 0 

Total number of duplicate rows for Geo Lookup: 0 

Total number of duplicate rows for Customers: 0 



In [8]:
# Verify Duplicate Orders Raw Data

df_orders_duplicates = df_orders_raw[df_orders_raw.duplicated(keep=False)]
print(df_orders_duplicates.head(10))
print("\n")

     USER_ID     ORDER_ID PURCHASE_TS     SHIP_TS DELIVERY_TS   REFUND_TS  \
10  30e358cc  b498734c2aa  12/27/2022  2022-12-28  2023-01-02         NaT   
11  30e358cc  b498734c2aa  12/27/2022  2022-12-28  2023-01-02         NaT   
19  f027c46b  29519934985    9/9/2021  2021-09-10  2021-09-18         NaT   
20  f027c46b  29519934985    9/9/2021  2021-09-10  2021-09-18         NaT   
27  d884c9ce  688e335ac1e    1/2/2019  2019-01-05  2019-01-09  2021-05-15   
28  d884c9ce  688e335ac1e    1/2/2019  2019-01-05  2019-01-09  2021-05-15   
32  9ea4fdb8  9d0da13842d  12/22/2022  2022-12-23  2022-12-31         NaT   
33  9ea4fdb8  9d0da13842d  12/22/2022  2022-12-23  2022-12-31         NaT   
38  7908c639  ffeec4c2097   1/20/2021  2021-01-23  2021-01-29         NaT   
39  7908c639  ffeec4c2097   1/20/2021  2021-01-23  2021-01-29         NaT   

       PRODUCT_NAME PRODUCT_ID  USD_PRICE  LOCAL_PRICE CURRENCY  \
10  ThinkPad Laptop       04ac     136.28      2598.00      TRY   
11  ThinkPad Lapto

Duplicate data was identified by seeing duplicate `user_id` and `order_id` combo. Therefore, we can perform a drop on the duplicates to ensure clean data.

In [9]:
# Drop duplicate rows
df_orders_clean = df_orders_raw.drop_duplicates(keep='first')

print(f"Total number of rows for df_orders_clean: {df_orders_clean.shape[0]}")

Total number of rows for df_orders_clean: 92931


In [12]:
# Create new table called orders_clean with table removed of duplicates
df_orders_clean.to_gbq(destination_table=f'{dataset_id}.orders_clean',
                       project_id='data-analysis-projects-456521',
                       if_exists='replace')

<ipython-input-12-c9f7c234b3f8>:2: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_orders_clean.to_gbq(destination_table=f'{dataset_id}.orders_clean',
100%|██████████| 1/1 [00:00<00:00, 2299.51it/s]


In [58]:
# Missing Data Checks
print(f"Total number of rows for Orders Clean: {df_orders_clean.shape[0]}")
orders_missing_data = df_orders_clean.isnull().sum()
print("Missing Orders Data: ")
print(orders_missing_data)
print("\n")

print(f"Total number of rows for Order Status: {df_order_status.shape[0]}")
order_status_missing_data = df_order_status.isnull().sum()
print("Missing Order Status Data: ")
print(order_status_missing_data)
print("\n")

print(f"Total number of rows for Geo Lookup: {df_geo_lookup.shape[0]}")
geo_lookup_missing_data = df_geo_lookup.isnull().sum()
print("Missing Geo Lookup Data: ")
print(geo_lookup_missing_data)
print("\n")

print(f"Total number of rows for Customers: {df_customers.shape[0]}")
customers_missing_data = df_customers.isnull().sum()
print("Missing Customers Data: ")
print(customers_missing_data)
print("\n")

Total number of rows for Orders Clean: 92931
Missing Orders Data: 
USER_ID                        0
ORDER_ID                       0
PURCHASE_TS                    2
SHIP_TS                        0
DELIVERY_TS                    0
REFUND_TS                  88231
PRODUCT_NAME                   0
PRODUCT_ID                     0
USD_PRICE                     28
LOCAL_PRICE                    0
CURRENCY                      51
PURCHASE_PLATFORM              0
MARKETING_CHANNEL           1186
ACCOUNT_CREATION_METHOD     1186
COUNTRY_CODE                 116
LOYALTY_PROGRAM                0
CREATED_ON                     0
dtype: int64


Total number of rows for Order Status: 1367
Missing Order Status Data: 
id             0
purchase_ts    0
ship_ts        0
delivery_ts    0
refund_ts      0
dtype: int64


Total number of rows for Geo Lookup: 189
Missing Geo Lookup Data: 
string_field_0    0
string_field_1    2
dtype: int64


Total number of rows for Customers: 74904
Missing Customers Dat

### Key Notes:
*   Table `geo_lookup` has incorrect/ missing field names
*   Missing data in field `REFUND_TS` on cleaned version of `orders_clean` is normal
*   Missing `COUNTRY_CODE` from `orders_clean` table can be possibly filled using other tables
*   Missing `MARKETING_CHANNEL` and `ACCOUNT_CREATION_METHOD` from `orders_clean` table can be possibly filled using other tables. This also applies vice versa

